In [ ]:
import numpy as np
import pandas as pd
import spacy
import re

In [ ]:
TRAIN_DATA = [
("Please take it once in a day thyroxine 25mg",{"entities":[(15,28,"frequency"),(29,38,"Medicine name"),(39,43,"dosage")]}),
("once in a day Thyroxine tablet 20mg ",{"entities":[(0,13,"frequency"),(14,23,"Medicine name"),(24,30,"type"),(31,35,"dosage")]}),
("Paracetamol tablet three times a day ",{"entities":[(0,11,"Medicine name"),(12,18,"type"),(19,36,"frequency")]}),
("Paracetamol tablet thrice a day ",{"entities":[(0,11,"Medicine name"),(12,18,"type"),(19,31,"frequency")]}),
("Adulsa syrup two times a day ",{"entities":[(0,6,"Medicine name"),(7,12,"type"),(14,29,"frequency")]}),
("Adulsa syrup twice a day ",{"entities":[(0,6,"Medicine name"),(7,12,"type"),(13,24,"frequency")]}),
("Crocin tablet once a day ",{"entities":[(0,6,"Medicine name"),(7,13,"type"),(14,24,"frequency")]}),
("Avas tablet 10mg twice a day ",{"entities":[(0,4,"Medicine name"),(5,11,"type"),(12,16,"dosage"),(17,28,"frequency")]}),
("Take 5ml of Benadryl syrup thrice for four days for dry cough.",{"entities":[(5,8,"dosage"),(12,20,"Medicine name"),(21,26,"type"),(27,33,"frequency"),(38,47,"period")]}),
("For dry cough take 5ml Alkof syrup thrice for five days.",{"entities":[(19,22,"dosage"),(23,28,"Medicine name"),(29,34,"type"),(35,41,"frequency"),(46,55,"period")]}),
("For a mild case of covid-19 like yours take HCQ tablet 400mg once for five days.",{"entities":[(44,47,"Medicine name"),(48,54,"type"),(55,60,"dosage"),(61,65,"frequency"),(70,79,"period")]}),
("since she has a mild fever I am prescribing crocin tablet 500mg twice a day for next three days.",{"entities":[(44,50,"Medicine name"),(51,57,"type"),(58,63,"dosage"),(64,75,"frequency"),(85,95,"period")]}),
("Shruti take ciplar 40mg tablet twice a day for a year to bring your condition of liver cirrhosis under control",{"entities":[(12,19,"Medicine name"),(19,23,"dosage"),(24,30,"type"),(31,42,"frequency"),(47,53,"period")]}),
("Take 3 times for 4 days for dry cough Benadryl syrup 5ml",{"entities":[(5,12,"frequency"),(17,23,"period"),(38,46,"Medicine name"),(47,52,"type"),(53,56,"dosage")]}),
("Take Benadryl syrup 5 ml 3 times for 4 days for dry cough",{"entities":[(5,13,"Medicine name"),(14,19,"type"),(20,24,"dosage"),(25,32,"frequency"),(36,43,"period")]}),
("Take Benadryl syrup 5 ml three times for four days for dry cough ",{"entities":[(5,14,"Medicine name"),(14,19,"type"),(20,24,"dosage"),(25,36,"frequency"),(41,50,"period")]}),
("Take Benadryl syrup 5 ml for four days for dry cough thrice",{"entities":[(5,13,"Medicine name"),(14,19,"type"),(20,24,"dosage"),(29,38,"period"),(53,59,"frequency")]}),
("For dry cough take 5ml Alkof syrup 2 times for five days.",{"entities":[(19,22,"dosage"),(23,28,"Medicine name"),(30,37,"frequency"),(42,51,"period")]}),
("For dry cough take 5ml Alkof syrup two times a day for five days.",{"entities":[(191,194,"dosage"),(215,220,"Medicine name"),(248,253,"type"),(272,287,"frequency"),(315,324,"period")]}),
("Take Benadryl syrup 5ml 3 times for 4 days for dry cough.",{"entities":[(21,24,"dosage"),(25,32,"frequency"),(37,43,"period"),(6,14,"Medicine name"),(14,19,"type")]}),
("Take crocin tablet 3 times in a day 500mg for next four days.",{"entities":[(5,11,"Medicine name"),(12,18,"type"),(19,35,"frequency"),(36,41,"dosage"),(51,60,"period")]}),
("Take crocin tablet two times in a day 500mg for next four days.",{"entities":[(8,14,"Medicine name"),(15,21,"type"),(22,33,"frequency"),(34,39,"dosage"),(49,58,"period")]}),
("To bring your condition of liver cirrhosis under control take 40mg tablet of ciplar twice a day for a year.",{"entities":[(62,66,"dosage"),(67,73,"type"),(77,83,"Medicine name"),(84,95,"frequency"),(100,106,"period")]}),
]

In [ ]:
spacy.__version__



'2.2.4'

In [ ]:
!mkdir "Named-Entity-Recognition"

mkdir: cannot create directory ‘Named-Entity-Recognition’: File exists


In [ ]:
# Load Packages
from __future__ import unicode_literals, print_function
import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm # loading bar

In [ ]:
model = None
output_dir=Path("Named-Entity-Recognition/model")
n_iter=200

In [ ]:
if model is not None:
    nlp1 = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp1 = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

Created blank 'en' model


In [ ]:
if 'ner' not in nlp1.pipe_names:
    ner = nlp1.create_pipe('ner')
    nlp1.add_pipe(ner, last=True)
    print('first')

else:
    ner = nlp1.get_pipe('ner')

first


In [ ]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

    # get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp1.pipe_names if pipe != 'ner']
with nlp1.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp1.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp1.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|██████████| 23/23 [00:01<00:00, 14.71it/s]


{'ner': 0.9667562004183792}


100%|██████████| 23/23 [00:01<00:00, 14.92it/s]


{'ner': 4.523035004611995}


100%|██████████| 23/23 [00:01<00:00, 14.76it/s]


...

{'ner': 11.473020779271122}


100%|██████████| 23/23 [00:01<00:00, 14.68it/s]


{'ner': 2.2195076091552184}


100%|██████████| 23/23 [00:01<00:00, 14.62it/s]

{'ner': 3.9963434002590263}


In [ ]:
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#https://www.enchantedlearning.com/wordlist/body.shtml

In [ ]:
more_stop=["pain","prescription","prescribe","prescribing","medicine","medication","surgery","operation","cut","dry","chest","lungs","liver",'hello','good','evening','morning','night','afternoon','r','c','b',"can't",'can','could','should','would','doctor','shall','will','maam',"ma'am","sir","hi","hey",'severe','mild','strong','slow','fast','high','low','pacing','blood pressure','sugar','blood','abdomen',"Adam's apple","adenoids","adrenal gland","anatomy","ankle","anus","appendix","arch","arm","artery","back","ball","of","the","foot","belly","belly","button","big","toe","bladder","blood","blood vessels","body","bone","brain","breast","buttocks","calf","capillary","carpal","cartilage","cell","cervical","vertebrae","cheek","chest","chin","circulatory","system","clavicle","coccyx","collar","bone","diaphragm","digestive","system"]
stop_words.extend(more_stop)



In [ ]:
stop_words.remove('a')

In [ ]:
text=["I am precribing a medicine called primaquine 10mg tablet twice a day for next four days"]


In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(text).str.replace("[^a-zA-Z-0-9]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
clean_sentences

['i am precribing a medicine called primaquine 10mg tablet twice a day for next four days']

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:


clean_sentences

['precribing a called primaquine 10mg tablet twice a day next four days']

In [ ]:
#Take Avas tablet 10mg twice a day for five days
doc = nlp1(clean_sentences[0])
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
output=[(ent.text, ent.label_) for ent in doc.ents]

Entities [('primaquine', 'Medicine name'), ('10mg', 'dosage'), ('tablet', 'type'), ('twice a day', 'frequency'), ('four days', 'period')]


In [ ]:

# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp1.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to Named-Entity-Recognition/model


In [ ]:

# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Loading from Named-Entity-Recognition/model
Entities [('crocin', 'Medicine name'), ('tablet', 'type'), ('500mg', 'dosage'), ('four days', 'period')]
Entities [('Benadryl', 'Medicine name'), ('syrup', 'type'), ('5 ml', 'dosage'), ('3 times', 'frequency'), ('4 days', 'period')]
Entities [('once in a day', 'frequency'), ('Thyroxine', 'Medicine name'), ('tablet', 'type'), ('20mg', 'dosage')]
Entities [('Benadryl', 'Medicine name'), ('syrup', 'type'), ('5 ml', 'dosage'), ('three times', 'frequency'), ('four days', 'period')]
Entities [('Adulsa', 'Medicine name'), ('syrup', 'type'), ('twice a day', 'frequency')]
Entities [('Benadryl', 'Medicine name'), ('syrup', 'type')]
Entities [('5ml', 'dosage'), ('Alkof', 'Medicine name'), ('syrup', 'type'), ('2 times', 'frequency'), ('five days', 'period')]
Entities [('ciplar', 'Medicine name'), ('40mg', 'dosage'), ('tablet', 'type'), ('twice a day', 'frequency'), ('a year', 'period')]
Entities [('Paracetamol', 'Medicine name'), ('tablet', 'type'), ('th

In [ ]:
len(TRAIN_DATA)

23

In [ ]:


single=[]
final=[]
for text, _ in TRAIN_DATA:
  doc = nlp2(text)
  single=[]
  for ent in doc.ents:
    single.append((ent.text,ent.label_))
  final.append(single)
print(final)
#print("Entities", [(ent.text, ent.label_) for ent in doc.ents])



[[('crocin', 'Medicine name'), ('tablet', 'type'), ('500mg', 'dosage'), ('four days', 'period')], [('Benadryl', 'Medicine name'), ('syrup', 'type'), ('5 ml', 'dosage'), ('3 times', 'frequency'), ('4 days', 'period')], [('once in a day', 'frequency'), ('Thyroxine', 'Medicine name'), ('tablet', 'type'), ('20mg', 'dosage')], [('Benadryl', 'Medicine name'), ('syrup', 'type'), ('5 ml', 'dosage'), ('three times', 'frequency'), ('four days', 'period')], [('Adulsa', 'Medicine name'), ('syrup', 'type'), ('twice a day', 'frequency')], [('Benadryl', 'Medicine name'), ('syrup', 'type')], [('5ml', 'dosage'), ('Alkof', 'Medicine name'), ('syrup', 'type'), ('2 times', 'frequency'), ('five days', 'period')], [('ciplar', 'Medicine name'), ('40mg', 'dosage'), ('tablet', 'type'), ('twice a day', 'frequency'), ('a year', 'period')], [('Paracetamol', 'Medicine name'), ('tablet', 'type'), ('thrice a day', 'frequency')], [('Adulsa', 'Medicine name'), ('syrup', 'type')], [('crocin', 'Medicine name'), ('tablet

In [ ]:

!apt-get install rar




Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  rar
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 295 kB of archives.
After this operation, 799 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 rar amd64 2:5.5.0-1 [295 kB]
Fetched 295 kB in 1s (357 kB/s)
Selecting previously unselected package rar.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../rar_2%3a5.5.0-1_amd64.deb ...
Unpacking rar (2:5.5.0-1) ...
Setting up rar (2:5.5.0-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
!rar a  "Named-Entity-Recognition.rar" "/content/Named-Entity-Recognition"


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive Named-Entity-Recognition.rar

Adding    /content/Named-Entity-Recognition/model/meta.json                0%  OK 
Adding    /content/Named-Entity-Recognition/model/tokenizer                2%  OK 
Adding    /content/Named-Entity-Recognition/model/vocab/key2row            2%  OK 
Adding    /content/Named-Entity-Recognition/model/vocab/vectors            2%  OK 
Adding    /content/Named-Entity-Recognition/model/vocab/strings.json       2%  OK 
Adding    /content/Named-Entity-Recognition/model/vocab/lexemes.bin        3%  OK 
Adding    /content/Named-Entity-Recognition/model/ner/moves                3%  OK 
Adding    /content/Named-Entity-Recognition/model/ner/cfg                  3%  OK 
Adding    /content/Named-Entity-Recognition/model/ner/model      

In [ ]:
#Example : Take Avas tablet 10mg twice a day for five days
for i in output:
  print(i)

('primaquine', 'Medicine name')
('10mg', 'dosage')
('tablet', 'type')
('twice a day', 'frequency')
('four days', 'period')


In [ ]:
!pip install FPDF
from fpdf import FPDF

In [ ]:
df=pd.DataFrame(columns=["Medicine name","type","dosage","frequency",'period'])

In [ ]:
first=[]
for i in output:
  first.append(i[0])

In [ ]:
df.loc[len(df)] = first

In [ ]:
df

,Medicine name,type,dosage,frequency,period
0,primaquine,10mg,tablet,twice a day,four days


In [ ]:
dr_details={
    'name':"Dr.Dushyant DAVE",
    "Qual":"M.B.B.S | ",
    "mobile":8291395170,
    "Address":"Bhavans Campus, Old D N Nagar, Munshi Nagar, Andheri West, Mumbai, Maharashtra 400058"
}

In [ ]:
patient_details={
    "name":"Vansh Jain",
    "age":21,
    "phone":8291395170,
    "Problem": "Fever"
}

In [ ]:
# Import FPDF class
from fpdf import FPDF

# Create instance of FPDF class

pdf=FPDF(format='letter', unit='in')

# Add new page.
pdf.add_page()

pdf.set_font('Times','',10.0)

# Effective page width, or just epw
epw = pdf.w - 2*pdf.l_margin

# Set column width to 1/5 of effective page width

col_width = epw/5



data = [first]



pdf.set_font('Times','B',14.0)
pdf.cell(0, txt=u'Dr. Dushyant',border=0, align='C')
pdf.set_font('Times','',10.0)
pdf.ln(0.2)
# Text height is the same as current font size
th = pdf.font_size
pdf.cell(0,th, txt=dr_details["Qual"],border=0, align='C')
pdf.ln(0.2)
pdf.cell(0,th, txt="Phone no: "+str(dr_details["mobile"]),border=0, align='C')
pdf.ln(0.2)
pdf.cell(0, th,txt=dr_details["Address"],border=0, align='C')
pdf.ln(0.4)
count=0
pdf.cell(col_width*5, 0, border='T')
pdf.ln(0.3)
pdf.cell(0,th, txt="Name: "+patient_details["name"],border=0, align='L')
pdf.cell(0,th, txt="Problem: "+str(patient_details["Problem"]),border=0, align='R')
pdf.ln(0.2)
pdf.cell(0,th, txt="Age: "+str(patient_details["age"]),border=0, align='L')
pdf.cell(0,th, txt="Phone no: "+str(patient_details["phone"]),border=0, align='R')
pdf.ln(0.4)
pdf.cell(col_width*5, 0, border='T')
pdf.ln(0.2)
pdf.set_font('Times','B',12.0)
th = pdf.font_size
for row in [["Medicine name","type","dosage","frequency",'period']]:
    for datum in row:
        # Enter data in column
        pdf.cell(col_width, 2*th, str(datum), border=1)

    pdf.ln(2*th)
pdf.set_font('Times','',10.0)
for row in data:
    for datum in row:
        # Enter data in column
        pdf.cell(col_width, 2*th, str(datum), border=1)

    pdf.ln(2*th)

pdf.output('table-using-cell-borders.pdf','F')

''

In [ ]:
pdf.output('table-using-cell-borders.pdf','F')

''